In [153]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse
import seaborn as sns
sns.set()
import re
import pickle

In [243]:
# with open('my_data.pkl', 'wb') as picklefile:
#     pickle.dump(data, picklefile)
    
# with open("my_data.pkl", 'rb') as picklefile: 
#     my_old_data = pickle.load(picklefile)

In [9]:
cnx = create_engine('postgresql://postgres@18.212.219.23:5432/postgres')
#cnx = create_engine('postgresql://username:password@ip_address:port/dbname')
#port ~ 5432

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [291]:
df=pd.read_sql_query('''SELECT * FROM biketrips ORDER BY RANDOM() LIMIT 100000''',cnx)

In [292]:
df.set_index('id', inplace=True, drop=False)

In [293]:
df.birthyear.isnull().sum()

682

### Start here if working from pickle
**"100k_sample.pkl"** Sample of 100,000 observations from bike trips, without geographic information  
**"manhattan.pkl"** ~11M rows of trips only taking place in manhattan, without user info, bike info, etc

In [294]:
#import pickle

# with open('100k_sample.pkl', 'wb') as picklefile:
#      pickle.dump(df, picklefile)

In [ ]:
with open("manhattan.pkl", 'rb') as picklefile: 
     df = pickle.load(picklefile)

In [ ]:
df=df.sample(n=100000, random_state=42)

In [298]:
usertypelist=df['usertype'].value_counts()

In [299]:
subscribers=usertypelist[0]
customers=usertypelist[1]

In [300]:
df.usertype.isnull().sum()

103

In [301]:
df.isnull().sum()

tripduration          0
starttime             0
stoptime              0
startstationid        0
startstationname      0
startstationlat       0
startstationlong      0
endstationid          0
endstationname        0
endstationlat         0
endstationlong        0
bikeid                0
usertype            103
birthyear           682
gender                0
id                    0
dtype: int64

In [302]:
#drop rows with no usertype information
df.dropna(axis='index', subset=['usertype'], inplace=True)

In [303]:
df.usertype.isnull().sum()

0

In [306]:
# convert NULL strings in "birthyear" to NoneType in order to convert birthyear col to float
df['birthyear']=df['birthyear'].replace(to_replace=['NULL'], value=None)
df.birthyear=df.birthyear.astype(float,copy=False)

In [307]:
#strip spaces in 'usertype'
df['usertype']=[x.strip() for x in df['usertype']]

In [308]:
#turn target categories into numbers: subscriber=0, customer=1
usermap={'Subscriber':0,'Customer':1}
df['target']=df.loc[:,'usertype'].map(usermap)

In [309]:
#create col for "presence of birthyear"  none provided=1, birth provided=0
def labelBirth(row):
    if row['birthyear']>0:
        return 0
    else:
        return 1
    
df['birthgiven']=df.apply(lambda row: labelBirth(row),axis=1)

In [310]:
#~600 customers gave birthyear (out of 11k)
mask = ((df["target"] == 1) & 
        (df["birthgiven"] == 1))
print("Percentage of customers who gave birthyear: " +str(int(len(df[mask])/customers*100)))

Percentage of customers who gave birthyear: 5


In [311]:
#~100 subscribers didn't give birthyear (out of 89k)
#~91 subscribers indicated unreasonable birth year (above age 80)
mask = ((df["target"] == 0) & 
        ((df["birthyear"]<1937)|
        df['birthyear'].isnull()))
print("Percentage of subscribers with no birth year or unlikely birthyear: " + str(round(len(df[mask])/customers*100, 2)))

Percentage of subscribers with no birth year or unlikely birthyear: 1.74


In [312]:
#~9k customers didn't gave gender (out of 11k)
#~1k subscribers didn't give gender (out of 89k)
# NOTE 0=unknown, 1=male, 2=female
mask = ((df["target"] == 1) & #customer
        (df["gender"] == 0))
print("Percentage of customers who did provide gender:" + str(int(100-(len(df[mask])/customers*100))))

mask = ((df["target"] == 0) & #subscriber
        (df["gender"] == 0))
print("Percent of subscribers who did provide gender: " +str(int(100-(len(df[mask])/subscribers*100))))

Percentage of customers who did provide gender:17
Percent of subscribers who did provide gender: 98


In [313]:
# create column indicating the hour in which the trip started
df['hour']=[x.hour for x in df['starttime']]

In [314]:
stationsgeo=pd.read_csv("selfstationsgeo.csv")
stationsgeo=stationsgeo.set_index(stationsgeo['id'])

In [315]:
def get_geos(df,stationsgeo):
    df['id']=df.index
    start=df[['startstationid']].join(stationsgeo[['hood', 'boro']], on='startstationid', how='left')
    tempdf=df.merge(start, left_index=True, right_index=True)
    tempdf.drop(columns=['startstationid_y'], inplace=True)
    startcolrenames={"startstationid_x":"startstationid","hood":"starthood",'boro':"startboro"}
    tempdf.rename(startcolrenames, axis="columns",inplace=True)
    
    df=tempdf
    
    end=df[['endstationid']].join(stationsgeo[['hood', 'boro']], on='endstationid', how='left')
    tempdf=df.merge(end, left_index=True, right_index=True)
    tempdf.drop(columns=['endstationid_y'], inplace=True)
    endcolrenames={"endstationid_x":"endstationid","hood":"endhood",'boro':"endboro"}
    tempdf.rename(endcolrenames, axis="columns",inplace=True)
    
    #for i in ['starthood','startboro','endhood','endboro']:
        #tempdf[i]=tempdf[i].fillna(value=0)
        
    tempdf=tempdf.drop(columns=['id'])
    
    return tempdf

In [316]:
df=get_geos(df,stationsgeo)

In [323]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'startstationid',
       'startstationname', 'startstationlat', 'startstationlong',
       'endstationid', 'endstationname', 'endstationlat', 'endstationlong',
       'bikeid', 'usertype', 'birthyear', 'gender', 'target', 'birthgiven',
       'hour', 'starthood', 'startboro', 'endhood', 'endboro'],
      dtype='object')

In [324]:
exportdf=df.drop(columns=['starttime', 'stoptime','startstationname', 'startstationlat', 'startstationlong','endstationname', 'endstationlat', 'endstationlong','bikeid', 'gender', 'birthyear', 'birthgiven', 'usertype'])

In [326]:
exportdf.columns

Index(['tripduration', 'startstationid', 'endstationid', 'target', 'hour',
       'starthood', 'startboro', 'endhood', 'endboro'],
      dtype='object')

In [327]:
len(exportdf)

99897

In [328]:
exportdf.dropna(axis=0,how='any',inplace=True)

In [329]:
len(exportdf)

54142

In [330]:
exportdf.to_csv("cleaned.csv")